<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setup-Database-Connection" data-toc-modified-id="Setup-Database-Connection-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup Database Connection</a></span><ul class="toc-item"><li><span><a href="#Get-last-record" data-toc-modified-id="Get-last-record-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Get last record</a></span></li></ul></li><li><span><a href="#Scrape-URLs" data-toc-modified-id="Scrape-URLs-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Scrape URLs</a></span><ul class="toc-item"><li><span><a href="#Collect-all-URLs-for-each-report" data-toc-modified-id="Collect-all-URLs-for-each-report-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Collect all URLs for each report</a></span></li><li><span><a href="#Sanity-Check" data-toc-modified-id="Sanity-Check-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Sanity Check</a></span></li></ul></li><li><span><a href="#Download,-Extract,-Import" data-toc-modified-id="Download,-Extract,-Import-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Download, Extract, Import</a></span></li><li><span><a href="#Compare-which-info-in-cpsc_contract_info-changes" data-toc-modified-id="Compare-which-info-in-cpsc_contract_info-changes-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Compare which info in cpsc_contract_info changes</a></span></li><li><span><a href="#Schema" data-toc-modified-id="Schema-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Schema</a></span></li><li><span><a href="#TODO" data-toc-modified-id="TODO-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>TODO</a></span></li></ul></div>

This is prototype for automatic scraping of one report type using methods from utils.py and db_operations.py

In [184]:
from bs4 import BeautifulSoup
import requests
import requests_cache

import pandas as pd
import numpy as np
import os
import shutil

from urllib.parse import urljoin, urlparse
from tqdm import tqdm

import time
import zipfile

import matplotlib.pyplot as plt
import seaborn as sns

from glob import glob
import re
import json

from datetime import datetime

In [185]:
# start requests cache so we can rurun the notebook without querying cms.gov
requests_cache.install_cache('cms_cache')

In [186]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [187]:
import utils
import table_schema
from db_operations import CRM_DB

In [188]:
# constants
DOWNLOADS_TABLE = 'meta_downloads'
DOWNLOAD_FILES_PATH = 'files'
START_URL = 'https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/MCRAdvPartDEnrolData/Monthly-Enrollment-by-Contract-Plan-State-County'

In [189]:
# display clickable url
print(START_URL)

https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/MCRAdvPartDEnrolData/Monthly-Enrollment-by-Contract-Plan-State-County


# Setup Database Connection
Pandas can write directly to an SQL database if given a SQLAlchemy engine or sqlite3 connection object.  Because we will later be using MySQL I will directly implement SQLAlchemy engine, but on a local sqlite db.

In [7]:
from sqlalchemy import create_engine
create_engine('sqlite:///test_db', echo=False)

Engine(sqlite:///test_db)

In [8]:
db = CRM_DB('sqlite:///test_db')

In [9]:
# db.metadata.reflect()
db.metadata.tables.keys()

dict_keys(['meta_downloads', 'cpsc_contract_info', 'cpsc_enrollment_info', 'monthly_report_by_contract'])

In [10]:
db.metadata.tables['meta_downloads']

Table('meta_downloads', MetaData(bind=Engine(sqlite:///test_db)), Column('ID', Integer(), table=<meta_downloads>, primary_key=True, nullable=False), Column('Table', String(), table=<meta_downloads>), Column('Prefix', String(), table=<meta_downloads>), Column('Filename', String(), table=<meta_downloads>), Column('URL', String(), table=<meta_downloads>), Column('Period', Date(), table=<meta_downloads>), schema=None)

## Get last record
TODO: Find a starting point, either by:
   1. querying a list of zip files that have been imported
   1. getting the last date of an imported record from the set.
  
To test this we need a fully constructed database so we will come back to it.

In [191]:
last_period = db.get_last_period('meta_downloads')
last_period

0

# Scrape URLs

## Collect all URLs for each report

Passing in last_period will acquire all download links, but only return downloads from after `last_period`

In [192]:
# get dataframes of all download links from oldest to newest
df = utils.get_all_downloads(START_URL, last_period=last_period, verbose=False)[::-1]

starting to extract urls
Extracting links from 100 rows
number of urls extrated 100
starting to extract urls
Extracting links from 81 rows
number of urls extrated 81
Accessing https://www.cms.gov/research-statistics-data-and-systemsstatistics-trends-and-reportsmcradvpartdenroldatamonthly/enrollment-contract-2021-06
--> recursion: 1
--> extracting downloads
Accessing https://www.cms.gov/research-statistics-data-and-systemsstatistics-trends-and-reportsmcradvpartdenroldatamonthly/monthly-enrollment-cpsc-2021-06
--> recursion: 1
--> extracting downloads
Accessing https://www.cms.gov/research-statistics-data-and-systemsstatistics-trends-and-reportsmcradvpartdenroldatamonthly/monthly-enrollment-cpsc-2021-05
--> recursion: 1
--> extracting downloads
Accessing https://www.cms.gov/research-statistics-data-and-systemsstatistics-trends-and-reportsmcradvpartdenroldatamonthly/monthly-enrollment-cpsc-2021-04
--> recursion: 1
--> extracting downloads
Accessing https://www.cms.gov/research-statistics-

--> extracting downloads
Accessing https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/MCRAdvPartDEnrolData/Monthly-Enrollment-by-Contract-Plan-State-County-Items/Monthly-Enrollment-by-CPSC-2018-03
--> recursion: 1
--> extracting downloads
Accessing https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/MCRAdvPartDEnrolData/Monthly-Enrollment-by-Contract-Plan-State-County-Items/Monthly-Enrollment-by-CPSC-2018-02
--> recursion: 1
--> extracting downloads
Accessing https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/MCRAdvPartDEnrolData/Monthly-Enrollment-by-Contract-Plan-State-County-Items/Monthly-Enrollment-by-CPSC-2018-01
--> recursion: 1
--> extracting downloads
Accessing https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/MCRAdvPartDEnrolData/Monthly-Enrollment-by-Contract-Plan-State-County-Items/Monthly-Enrollment-by-CPSC-2017-12
--> recursion: 1


--> extracting downloads
Accessing https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/MCRAdvPartDEnrolData/Monthly-Enrollment-by-Contract-Plan-State-County-Items/Monthly-Enrollment-by-CPSC-2015-06
--> recursion: 1
--> extracting downloads
Accessing https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/MCRAdvPartDEnrolData/Monthly-Enrollment-by-Contract-Plan-State-County-Items/Monthly-Enrollment-by-CPSC-2015-05
--> recursion: 1
--> extracting downloads
Accessing https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/MCRAdvPartDEnrolData/Monthly-Enrollment-by-Contract-Plan-State-County-Items/Monthly-Enrollment-by-CPSC-2015-04
--> recursion: 1
--> extracting downloads
Accessing https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/MCRAdvPartDEnrolData/Monthly-Enrollment-by-Contract-Plan-State-County-Items/Monthly-Enrollment-by-CPSC-2015-03
--> recursion: 1


Accessing https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/MCRAdvPartDEnrolData/Monthly-Enrollment-by-Contract-Plan-State-County-Items/Monthly-Enrollment-by-CPSC-2012-06
--> recursion: 1
--> extracting downloads
Accessing https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/MCRAdvPartDEnrolData/Monthly-Enrollment-by-Contract-Plan-State-County-Items/Monthly-Enrollment-by-CPSC-2012-05
--> recursion: 1
--> extracting downloads
Accessing https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/MCRAdvPartDEnrolData/Monthly-Enrollment-by-Contract-Plan-State-County-Items/Monthly-Enrollment-by-CPSC
--> recursion: 1
--> extracting downloads
Accessing https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/MCRAdvPartDEnrolData/Monthly-Enrollment-by-Contract-Plan-State-County-Items/CMS1256746
--> recursion: 1
--> extracting downloads
Accessing https://www.cms.gov/Re

Accessing https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/MCRAdvPartDEnrolData/Monthly-Enrollment-by-Contract-Plan-State-County-Items/CMS1221839
--> recursion: 1
--> extracting downloads
Accessing https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/MCRAdvPartDEnrolData/Monthly-Enrollment-by-Contract-Plan-State-County-Items/CMS1221060
--> recursion: 1
--> extracting downloads
Accessing https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/MCRAdvPartDEnrolData/Monthly-Enrollment-by-Contract-Plan-State-County-Items/CMS1220465
--> recursion: 1
--> extracting downloads
Accessing https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/MCRAdvPartDEnrolData/Monthly-Enrollment-by-Contract-Plan-State-County-Items/CMS1223265
--> recursion: 1
--> extracting downloads
Accessing https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports

In [193]:
df

,report_period,report_title,download_url,download_page_url,description,page_title,page_url
0,2006-07,Monthly Enrollment by CPSC - July 2006 (ZIP),/Research-Statistics-Data-and-Systems/Statisti...,https://www.cms.gov/Research-Statistics-Data-a...,Provides monthly enrollment at the contract/pl...,Monthly Enrollment by Contract/Plan/State/County,https://www.cms.gov/Research-Statistics-Data-a...
0,2006-08,Monthly Enrollment by CPSC - August 2006 (ZIP),/Research-Statistics-Data-and-Systems/Statisti...,https://www.cms.gov/Research-Statistics-Data-a...,Provides monthly enrollment at the contract/pl...,Monthly Enrollment by Contract/Plan/State/County,https://www.cms.gov/Research-Statistics-Data-a...
0,2006-09,Monthly Enrollment by CPSC - Sept 2006 (ZIP),/Research-Statistics-Data-and-Systems/Statisti...,https://www.cms.gov/Research-Statistics-Data-a...,Provides monthly enrollment at the contract/pl...,Monthly Enrollment by Contract/Plan/State/County,https://www.cms.gov/Research-Statistics-Data-a...
0,2006-10,Monthly Enrollment by CPSC - October 2006 (ZIP),/Research-Statistics-Data-and-Systems/Statisti...,https://www.cms.gov/Research-Statistics-Data-a...,Provides monthly enrollment at the contract/pl...,Monthly Enrollment by Contract/Plan/State/County,https://www.cms.gov/Research-Statistics-Data-a...
0,2006-11,Monthly Enrollment by CPSC - November 2006 (ZIP),/Research-Statistics-Data-and-Systems/Statisti...,https://www.cms.gov/Research-Statistics-Data-a...,Provides monthly enrollment at the contract/pl...,Monthly Enrollment by Contract/Plan/State/County,https://www.cms.gov/Research-Statistics-Data-a...
...,...,...,...,...,...,...,...
0,2021-03,Monthly Enrollment by CPSC – March 2021 (ZIP),/files/zip/monthly-enrollment-cpsc-march-2021.zip,https://www.cms.gov/research-statistics-data-a...,Provides monthly enrollment at the contract/pl...,Monthly Enrollment by Contract/Plan/State/County,https://www.cms.gov/Research-Statistics-Data-a...
0,2021-04,Monthly Enrollment by CPSC – April 2021 (ZIP),/files/zip/monthly-enrollment-cpsc-april-2021.zip,https://www.cms.gov/research-statistics-data-a...,Provides monthly enrollment at the contract/pl...,Monthly Enrollment by Contract/Plan/State/County,https://www.cms.gov/Research-Statistics-Data-a...
0,2021-05,Monthly Enrollment by CPSC – May 2021 (ZIP),/files/zip/monthly-enrollment-cpsc-may-2021.zip,https://www.cms.gov/research-statistics-data-a...,Provides monthly enrollment at the contract/pl...,Monthly Enrollment by Contract/Plan/State/County,https://www.cms.gov/Research-Statistics-Data-a...
0,2021-06,Monthly Enrollment by CPSC – June 2021 (ZIP),/files/zip/monthly-enrollment-cpsc-june-2021.zip,https://www.cms.gov/research-statistics-data-a...,Provides monthly enrollment at the contract/pl...,Monthly Enrollment by Contract/Plan/State/County,https://www.cms.gov/Research-Statistics-Data-a...


## Sanity Check

In [194]:
# Unique page urls == 1
df['page_url'].nunique()

1

In [195]:
# Unique page titles = 1
df['page_title'].nunique()

1

In [196]:
# num download urls (total files to download)
df['download_url'].nunique()

181

In [197]:
soup = utils.get_page_soup(START_URL)
utils.extract_num_entries(soup)

181

In [198]:
df.shape

(181, 7)

# Download, Extract, Import

In [190]:
os.remove('test_db')
db = CRM_DB('sqlite:///test_db')


In [199]:
start_time = datetime.now()

# loop through files in each zip

# TODO: temporarily limit to 2
for idx, row in df.iterrows():
    
    print(row)
    
    period = pd.to_datetime(row['report_period'])
    
    # download file
    url = urljoin(START_URL, row['download_url'])
    prefix, filename = utils.download_file(url, DOWNLOAD_FILES_PATH)
    
    zip_file_meta = {'Prefix': prefix,
                 'Filename': filename,
                 'URL': url,
                 'Period': pd.to_datetime(row['report_period']),
                 }
    
    # extract zip file to temp_dir
    zipfile_path = os.path.join(DOWNLOAD_FILES_PATH, prefix, filename)
    tmp_dir='tmp_dir' 
    utils.extract_zip(zipfile_path, tmp_dir=tmp_dir)
    
    # get list of all unzipped files
    files = glob(os.path.join(tmp_dir, '**/*.*'), recursive=True)
    #print(files)
    
    # parse tables and filenames
    parsed_files = utils.parse_files_by_table(files)
    display(parsed_files)
    
    updated_tables = []
    # loop through files and store in db
    for table, file in parsed_files.items():
        
        
        # create record in meta_downloads for this insert
        zip_file_meta['Table'] = table
        meta_downloads_id = db.insert_dict(DOWNLOADS_TABLE, zip_file_meta)
        print('Zip file row created with id', meta_downloads_id)
        
        # load the file into a df
        _, ext = os.path.splitext(file)
        if ext == '.csv':
            file_df = pd.read_csv(file, encoding='latin1')
        else:
            raise Exception(f'File type "{ext}" not handled')
        
        print('insert shape:', file_df.shape)
        print('Time Elapsed:', datetime.now()-start_time)
        
        file_df = utils.prepare_table_for_insert(file_df, period)
        print('DataFrame prepared for insert')
        print('Time Elapsed:', datetime.now()-start_time)
        
        
        # TODO - check if the table rows should be inserted or updated and act.
        
        db.insert_df(table, file_df)
        print('inserted performed on ', table)
        print('Time Elapsed:', datetime.now()-start_time)
        
        # update meta_downloads to indicate success
        db.mark_success(meta_downloads_id)

    print('Time Elapsed:', datetime.now()-start_time)
    # remove extracted files
    shutil.rmtree(tmp_dir)

    # shutil.rmtree is SLOW
#     os.system(f'rm -fr "{tmp_dir}"')

    print('folder removed')
    print('Time Elapsed:', datetime.now()-start_time)

    
print('Time Elapsed:', datetime.now()-start_time)

report_period                                                  2006-07
report_title              Monthly Enrollment by CPSC - July 2006 (ZIP)
download_url         /Research-Statistics-Data-and-Systems/Statisti...
download_page_url    https://www.cms.gov/Research-Statistics-Data-a...
description          Provides monthly enrollment at the contract/pl...
page_title            Monthly Enrollment by Contract/Plan/State/County
page_url             https://www.cms.gov/Research-Statistics-Data-a...
Name: 0, dtype: object
['tmp_dir/CPSC_Contract_Info_2006_07.csv', 'tmp_dir/CPSC_Enrollment_Info_2006_07.csv', 'tmp_dir/Read_Me_CPSC_Enrollment_2006.txt']


{'cpsc_contract_info': 'tmp_dir/CPSC_Contract_Info_2006_07.csv',
 'cpsc_enrollment_info': 'tmp_dir/CPSC_Enrollment_Info_2006_07.csv'}

Zip file row created with id 1
insert shape: (4924, 12)
Time Elapsed: 0:00:02.663007
DataFrame prepared for insert
Time Elapsed: 0:00:02.899689
inserted performed on  cpsc_contract_info
Time Elapsed: 0:00:03.236572
Zip file row created with id 2
insert shape: (1227922, 7)
Time Elapsed: 0:00:04.797737
DataFrame prepared for insert
Time Elapsed: 0:00:06.616601
inserted performed on  cpsc_enrollment_info
Time Elapsed: 0:00:33.494519
Time Elapsed: 0:00:33.501802
folder removed
Time Elapsed: 0:00:33.502445
report_period                                                  2006-08
report_title            Monthly Enrollment by CPSC - August 2006 (ZIP)
download_url         /Research-Statistics-Data-and-Systems/Statisti...
download_page_url    https://www.cms.gov/Research-Statistics-Data-a...
description          Provides monthly enrollment at the contract/pl...
page_title            Monthly Enrollment by Contract/Plan/State/County
page_url             https://www.cms.gov/Research-Statistics-Data-a

{'cpsc_contract_info': 'tmp_dir/CPSC_Contract_Info_2006_08.csv',
 'cpsc_enrollment_info': 'tmp_dir/CPSC_Enrollment_Info_2006_08.csv'}

Zip file row created with id 3
insert shape: (4923, 12)
Time Elapsed: 0:00:34.782747
DataFrame prepared for insert
Time Elapsed: 0:00:34.809162
inserted performed on  cpsc_contract_info
Time Elapsed: 0:00:35.316051
Zip file row created with id 4
insert shape: (1235096, 7)
Time Elapsed: 0:00:36.714036
DataFrame prepared for insert
Time Elapsed: 0:00:39.108306
inserted performed on  cpsc_enrollment_info
Time Elapsed: 0:01:31.424072
Time Elapsed: 0:01:31.466928
folder removed
Time Elapsed: 0:01:31.467620
report_period                                                  2006-09
report_title              Monthly Enrollment by CPSC - Sept 2006 (ZIP)
download_url         /Research-Statistics-Data-and-Systems/Statisti...
download_page_url    https://www.cms.gov/Research-Statistics-Data-a...
description          Provides monthly enrollment at the contract/pl...
page_title            Monthly Enrollment by Contract/Plan/State/County
page_url             https://www.cms.gov/Research-Statistics-Data-a

...

{'cpsc_contract_info': 'tmp_dir/CPSC_Contract_Info_2007_12.csv',
 'cpsc_enrollment_info': 'tmp_dir/CPSC_Enrollment_Info_2007_12.csv'}

Zip file row created with id 35
insert shape: (5859, 12)
Time Elapsed: 8:37:18.527115
DataFrame prepared for insert
Time Elapsed: 8:37:19.333831
inserted performed on  cpsc_contract_info
Time Elapsed: 8:37:33.632935
Zip file row created with id 36
insert shape: (1491356, 7)
Time Elapsed: 8:37:37.404743
DataFrame prepared for insert
Time Elapsed: 8:37:39.627677
inserted performed on  cpsc_enrollment_info
Time Elapsed: 9:50:31.612599
Time Elapsed: 9:50:31.657071
folder removed
Time Elapsed: 9:50:31.703945
report_period                                                  2008-01
report_title           Monthly Enrollment by CPSC - January 2008 (ZIP)
download_url         /Research-Statistics-Data-and-Systems/Statisti...
download_page_url    https://www.cms.gov/Research-Statistics-Data-a...
description          Provides monthly enrollment at the contract/pl...
page_title            Monthly Enrollment by Contract/Plan/State/County
page_url             https://www.cms.gov/Research-Statistics-Data

{'cpsc_contract_info': 'tmp_dir/CPSC_Contract_Info_2008_01.csv',
 'cpsc_enrollment_info': 'tmp_dir/CPSC_Enrollment_Info_2008_01.csv',
 'readme_cpsc_enrollment': 'tmp_dir/readme_CPSC_enrollment_2008.txt'}

Zip file row created with id 37
insert shape: (6839, 12)
Time Elapsed: 9:50:41.529769
DataFrame prepared for insert
Time Elapsed: 9:50:42.731893
inserted performed on  cpsc_contract_info
Time Elapsed: 9:50:54.468286
Zip file row created with id 38
insert shape: (1786610, 7)
Time Elapsed: 9:50:56.031169
DataFrame prepared for insert
Time Elapsed: 9:50:57.740511
inserted performed on  cpsc_enrollment_info
Time Elapsed: 11:00:25.634716
Zip file row created with id 39


Exception: File type ".txt" not handled

In [156]:
downloads = db.query('SELECT * FROM meta_downloads ORDER BY id DESC LIMIT 100')
downloads

,ID,Table,Prefix,Filename,URL,Period,Succsess
0,7,cpsc_enrollment_info,cpsc_enrollment,CPSC_Enrollment_2021_06.zip,https://www.cms.gov/files/zip/monthly-enrollme...,2021-06-01,1
1,6,cpsc_contract_info,cpsc_enrollment,CPSC_Enrollment_2021_06.zip,https://www.cms.gov/files/zip/monthly-enrollme...,2021-06-01,1
2,5,monthly_report_by_contract,monthly_report_by_contract,Monthly_Report_By_Contract_2021_06.zip,https://www.cms.gov/files/zip/monthly-enrollme...,2021-06-01,1
3,4,cpsc_enrollment_info,cpsc_enrollment,CPSC_Enrollment_2006_09.zip,https://www.cms.gov/Research-Statistics-Data-a...,2006-09-01,1
4,3,cpsc_contract_info,cpsc_enrollment,CPSC_Enrollment_2006_09.zip,https://www.cms.gov/Research-Statistics-Data-a...,2006-09-01,1
5,2,cpsc_enrollment_info,cpsc_enrollment,CPSC_Enrollment_2006_08.zip,https://www.cms.gov/Research-Statistics-Data-a...,2006-08-01,1
6,1,cpsc_contract_info,cpsc_enrollment,CPSC_Enrollment_2006_08.zip,https://www.cms.gov/Research-Statistics-Data-a...,2006-08-01,1


In [92]:
db.query('SELECT * FROM cpsc_enrollment_info ORDER BY id DESC LIMIT 100')

,ID,ContractNumber,PlanID,SSAStateCountyCode,FIPSStateCountyCode,State,County,Enrollment,Period
0,1235096,S9264,802,53220,56045,WY,Weston,5,2006-08-01 00:00:00.000000
1,1235095,S9264,802,53210,56043,WY,Washakie,5,2006-08-01 00:00:00.000000
2,1235094,S9264,802,53200,56041,WY,Uinta,5,2006-08-01 00:00:00.000000
3,1235093,S9264,802,53190,56039,WY,Teton,5,2006-08-01 00:00:00.000000
4,1235092,S9264,802,53180,56037,WY,Sweetwater,5,2006-08-01 00:00:00.000000
...,...,...,...,...,...,...,...,...,...
95,1235001,S9264,802,51540,54109,WV,Wyoming,5,2006-08-01 00:00:00.000000
96,1235000,S9264,802,51530,54107,WV,Wood,5,2006-08-01 00:00:00.000000
97,1234999,S9264,802,51520,54105,WV,Wirt,5,2006-08-01 00:00:00.000000
98,1234998,S9264,802,51510,54103,WV,Wetzel,5,2006-08-01 00:00:00.000000


In [93]:
db.query('SELECT * FROM cpsc_contract_info ORDER BY id DESC LIMIT 100')

,ID,ContractID,PlanID,OrganizationType,PlanType,OffersPartD,SNPPlan,EGHP,OrganizationName,OrganizationMarketingName,PlanName,ParentOrganization,ContractEffectiveDate,Period
0,4923,S9264,802,Medicare Prescription Drug Plan,Medicare Prescription Drug Plan,1,0,1,STERLING LIFE INSURANCE COMPANY,Sterling Retiree Rx,Sterling Retiree Rx,Sterling Insurance Group,01/01/2006 0:00:00,2006-08-01 00:00:00.000000
1,4922,S9264,801,Medicare Prescription Drug Plan,Medicare Prescription Drug Plan,1,0,1,STERLING LIFE INSURANCE COMPANY,Sterling Retiree Rx,Sterling Retiree Rx,Sterling Insurance Group,01/01/2006 0:00:00,2006-08-01 00:00:00.000000
2,4921,S9086,1,Medicare Prescription Drug Plan,Medicare Prescription Drug Plan,1,0,0,AMERICA'S HEALTHCHOICE,America's Health Choice,AHC Prescription Drug Plan,"America's Health Choice Medical Plans, Inc",01/01/2006 0:00:00,2006-08-01 00:00:00.000000
3,4920,S8067,802,Medicare Prescription Drug Plan,Medicare Prescription Drug Plan,1,0,1,AVALON INSURANCE COMPANY,Avalon Insurance Company,SecureRx - ER - Non-Calendar PY,Capital BlueCross,01/01/2006 0:00:00,2006-08-01 00:00:00.000000
4,4919,S8067,801,Medicare Prescription Drug Plan,Medicare Prescription Drug Plan,1,0,1,AVALON INSURANCE COMPANY,Avalon Insurance Company,SecureRx - ER,Capital BlueCross,01/01/2006 0:00:00,2006-08-01 00:00:00.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,4828,S5967,60,Medicare Prescription Drug Plan,Medicare Prescription Drug Plan,1,0,0,"WELLCARE PRESCRIPTION INSURANCE, INC.",WellCare,WellCare Signature,"WellCare Health Plans, Inc.",01/01/2006 0:00:00,2006-08-01 00:00:00.000000
96,4827,S5967,59,Medicare Prescription Drug Plan,Medicare Prescription Drug Plan,1,0,0,"WELLCARE PRESCRIPTION INSURANCE, INC.",WellCare,WellCare Signature,"WellCare Health Plans, Inc.",01/01/2006 0:00:00,2006-08-01 00:00:00.000000
97,4826,S5967,58,Medicare Prescription Drug Plan,Medicare Prescription Drug Plan,1,0,0,"WELLCARE PRESCRIPTION INSURANCE, INC.",WellCare,WellCare Signature,"WellCare Health Plans, Inc.",01/01/2006 0:00:00,2006-08-01 00:00:00.000000
98,4825,S5967,57,Medicare Prescription Drug Plan,Medicare Prescription Drug Plan,1,0,0,"WELLCARE PRESCRIPTION INSURANCE, INC.",WellCare,WellCare Signature,"WellCare Health Plans, Inc.",01/01/2006 0:00:00,2006-08-01 00:00:00.000000


In [ ]:
db.metadata.tables['meta_downloads']

# Compare which info in cpsc_contract_info changes

In [141]:
aug = db.query('SELECT * FROM cpsc_contract_info WHERE Period = "2006-08-01 00:00:00.000000"')
aug.head()

,ID,ContractID,PlanID,OrganizationType,PlanType,OffersPartD,SNPPlan,EGHP,OrganizationName,OrganizationMarketingName,PlanName,ParentOrganization,ContractEffectiveDate,Period
0,1,90091,NaN,HCPP - 1833 Cost,HCPP - 1833 Cost,0,0,0,UNITED MINE WORKERS OF AMERICA,United Mine Workers of America,None,None,02/01/1974 0:00:00,2006-08-01 00:00:00.000000
1,2,90400,NaN,Pilot,Pilot,0,0,0,CIGNA MEDICARE HEALTH SUPPORT,Cigna Healthcare,None,CIGNA,09/01/2005 0:00:00,2006-08-01 00:00:00.000000
2,3,90401,NaN,Pilot,Pilot,0,0,0,XL HEALTH MEDICARE HEALTH SUPPORT,Xl Health Corporation,None,None,11/01/2005 0:00:00,2006-08-01 00:00:00.000000
3,4,90402,NaN,Pilot,Pilot,0,0,0,AMERICAN HEALTHWAYS MEDICARE HEALTH SUPPORT,Healthways,None,"American Healthways, Inc.",08/01/2005 0:00:00,2006-08-01 00:00:00.000000
4,5,90403,NaN,Pilot,Pilot,0,0,0,MCKESSON MEDICARE HEALTH SUPPORT,Mckesson Health Solutions,None,McKesson Health Solutions,08/01/2005 0:00:00,2006-08-01 00:00:00.000000


In [157]:
all_months = db.query('SELECT * FROM cpsc_contract_info WHERE Period > "2006-08-00 00:00:00.000000"')
all_months.head()

,ID,ContractID,PlanID,OrganizationType,PlanType,OffersPartD,SNPPlan,EGHP,OrganizationName,OrganizationMarketingName,PlanName,ParentOrganization,ContractEffectiveDate,Period
0,1,90091,NaN,HCPP - 1833 Cost,HCPP - 1833 Cost,0,0,0,UNITED MINE WORKERS OF AMERICA,United Mine Workers of America,None,None,02/01/1974 0:00:00,2006-08-01 00:00:00.000000
1,2,90400,NaN,Pilot,Pilot,0,0,0,CIGNA MEDICARE HEALTH SUPPORT,Cigna Healthcare,None,CIGNA,09/01/2005 0:00:00,2006-08-01 00:00:00.000000
2,3,90401,NaN,Pilot,Pilot,0,0,0,XL HEALTH MEDICARE HEALTH SUPPORT,Xl Health Corporation,None,None,11/01/2005 0:00:00,2006-08-01 00:00:00.000000
3,4,90402,NaN,Pilot,Pilot,0,0,0,AMERICAN HEALTHWAYS MEDICARE HEALTH SUPPORT,Healthways,None,"American Healthways, Inc.",08/01/2005 0:00:00,2006-08-01 00:00:00.000000
4,5,90403,NaN,Pilot,Pilot,0,0,0,MCKESSON MEDICARE HEALTH SUPPORT,Mckesson Health Solutions,None,McKesson Health Solutions,08/01/2005 0:00:00,2006-08-01 00:00:00.000000


In [158]:
all_months[all_months.duplicated(subset=['ContractID', 'PlanID'], keep=False)].shape

(10651, 14)

In [167]:
duplicates = all_months[all_months.duplicated(subset=['ContractID', 'PlanID'], keep=False)]
duplicates.sort_values(by=['ContractID', 'PlanID']).iloc[:3]

,ID,ContractID,PlanID,OrganizationType,PlanType,OffersPartD,SNPPlan,EGHP,OrganizationName,OrganizationMarketingName,PlanName,ParentOrganization,ContractEffectiveDate,Period
0,1,90091,NaN,HCPP - 1833 Cost,HCPP - 1833 Cost,0,0,0,UNITED MINE WORKERS OF AMERICA,United Mine Workers of America,None,None,02/01/1974 0:00:00,2006-08-01 00:00:00.000000
4923,4924,90091,NaN,HCPP - 1833 Cost,HCPP - 1833 Cost,0,0,0,UNITED MINE WORKERS OF AMERICA,United Mine Workers of America,None,None,02/01/1974 0:00:00,2006-09-01 00:00:00.000000
9846,9847,90091,NaN,HCPP - 1833 Cost,HCPP - 1833 Cost,0,0,0,UNITED MINE WORKERS OF AMERICA HLTH & RETIREMENT,United Mine Workers of America Health & Retire...,None,UMWA Health and Retirement Funds,02/01/1974,2021-06-01 00:00:00.000000


In [178]:
duplicates.columns

Index(['ID', 'ContractID', 'PlanID', 'OrganizationType', 'PlanType',
       'OffersPartD', 'SNPPlan', 'EGHP', 'OrganizationName',
       'OrganizationMarketingName', 'PlanName', 'ParentOrganization',
       'ContractEffectiveDate', 'Period'],
      dtype='object')

In [183]:
duplicates[duplicates.duplicated(subset=['OffersPartD', 'SNPPlan', 'EGHP'], keep=False)].sort_values(by=['ContractID', 'PlanID']).shape

(10651, 14)

# Schema


In [ ]:
from eralchemy import render_er
## Draw from SQLAlchemy base
#render_er(db.engine, 'erd_from_sqlalchemy.png')

In [ ]:
## Draw from database
render_er("sqlite:///test_db", 'erd_from_sqlite.png')

# TODO

* x ensure name in download table can be matched to name in data table
* Do cpsc_contract_info monthly_report_by_contract need to be updated rather than added to?
    Looks like yes, some information changes, but some plans may also be removed.
* Switch to MySQL locally to easily handle updates
* Handle txt files in this batch of files.
* Consider deploying a Docker container on ECS 


In [129]:
import sqlite3

In [130]:
con = sqlite3.connect('test_db')

In [133]:
cursor = con.cursor()

In [137]:
cursor.execute('SELECT * FROM cpsc_contract_info WHERE Period = "2006-08-01 00:00:00.000000"  LIMIT 10')

In [138]:
cursor.fetchall()

[(1,
  '90091',
  None,
  'HCPP - 1833 Cost',
  'HCPP - 1833 Cost',
  0,
  0,
  0,
  'UNITED MINE WORKERS OF AMERICA',
  'United Mine Workers of America',
  None,
  None,
  '02/01/1974 0:00:00',
  '2006-08-01 00:00:00.000000'),
 (2,
  '90400',
  None,
  'Pilot',
  'Pilot',
  0,
  0,
  0,
  'CIGNA MEDICARE HEALTH SUPPORT',
  'Cigna Healthcare',
  None,
  'CIGNA',
  '09/01/2005 0:00:00',
  '2006-08-01 00:00:00.000000'),
 (3,
  '90401',
  None,
  'Pilot',
  'Pilot',
  0,
  0,
  0,
  'XL HEALTH MEDICARE HEALTH SUPPORT',
  'Xl Health Corporation',
  None,
  None,
  '11/01/2005 0:00:00',
  '2006-08-01 00:00:00.000000'),
 (4,
  '90402',
  None,
  'Pilot',
  'Pilot',
  0,
  0,
  0,
  'AMERICAN HEALTHWAYS MEDICARE HEALTH SUPPORT',
  'Healthways',
  None,
  'American Healthways, Inc.',
  '08/01/2005 0:00:00',
  '2006-08-01 00:00:00.000000'),
 (5,
  '90403',
  None,
  'Pilot',
  'Pilot',
  0,
  0,
  0,
  'MCKESSON MEDICARE HEALTH SUPPORT',
  'Mckesson Health Solutions',
  None,
  'McKesson Health